In [1]:
# 라이브러리 호출
import bok_da as bd
import numpy as np
import pandas as pd
import scipy.stats as sp
import matplotlib.pyplot as plt
import seaborn as sns

#### 본 매뉴얼은 그레인져 인과관계 검정에 대해 다룬다.
- 작성자: 고려대학교 경제학과 강규호 교수, 디지털신기술팀 이창훈 과장

## 1. 데이터 불러오기

데이터는 한국의 전기대비 실질 GDP 성장률, CD91일 금리, 인플레이션율 소비자 가격 지수 (Consumer Price Index, CPI), CD91일 금리로 1999년 4분기부터 2023년 4분끼지의 분기별 자료이다. 해당 데이터는 ECOS (https://ecos.bok.or.kr/#/)에서 다운로드 받았으며 컴퓨터에 "python_data.xlsx"라는 이름의 엑셀파일의 "KORmacro"라는 시트에 저장되어 있다. 이때, 인플레이션율의 경우 소비자 가격 지수 (Consumer Price Index, CPI)의 전년동기대비 치분 (YoY)을 통해 미리 변환한 것이다.

`data` 폴더에서 `python_data.xlsx`를 불러온 후 파이썬 표준 자료형인 `pandas.DataFrame`으로 전처리를 수행한다. `DataFrame`은 테이블 형태의 데이터의 행(날짜)과 열(변수명) 정보를 가지고 있어, 행렬이나 벡터형태로 분석하는 것보다 유용하다. `df`라는 이름으로 할당하는 것이 일반적이다. 데이터프레임에 익숙하지 않은 경우 아래 셀의 코드를 줄별로 여러 셀로 나누어 실행해보는 것을 추천한다. 원천기관에서 raw 데이터를 불러오고 전처리하는 과정이 복잡해 보일 수 있지만, 한번 해두면 크게 코드를 수정할 필요없이 새로운 데이터에 대해서도 분석 가능한 형태로 전처리 할 수 있기 때문에 장기적으로 효율적인 방법이다.

In [2]:
df = pd.read_excel('data/python_data.xlsx', sheet_name='KORmacro') # 데이터 엑셀파일 불러오기
df = df.set_index('date') # date 명의 열을 인덱스로 지정한다.
df.index.name = None # 인덱스 이름 date를 제거한다.
df.index = df.index.str.replace('/', '-') # 2000/03 형식을 pandas가 날짜로 인식하도록 2000-3형식으로 변환한다.
df.index = pd.to_datetime(df.index) + pd.offsets.QuarterEnd(0) # 인덱스(행)를 날짜로 인식하도록 변환한다.
df.index.freq = pd.offsets.QuarterEnd() # 인덱스의 frequency를 분기말(2000-03-31)로 설정한다.
column_order = ['inf', 'rgdp', 'cd91'] # df의 열 순서를 리스트로 미리 할당
df = df[column_order] # df의 열 순서를 column_order에 맞게 변경
df

,inf,rgdp,cd91
2000-12-31,2.508257,-0.3,6.97
2001-03-31,3.627876,1.2,5.95
2001-06-30,4.919778,1.3,5.78
2001-09-30,4.158456,1.4,4.97
2001-12-31,3.244327,1.8,4.58
...,...,...,...
2022-12-31,5.083054,-0.3,3.91
2023-03-31,4.496285,0.3,3.64
2023-06-30,3.212650,0.6,3.63
2023-09-30,3.079117,0.6,3.74


## 2. 그레인저 인과관계 검정 (Granger Causality Test)

#### 구문
`bok_da` 라이브러리-시계열분석 패키지(`ts`)-시계열검정 서브패키지(`test`)의 `granger_causality_test`라는 함수를 사용하면 y1과 y2 벡터의 그레인저 인과관계 검정 결과를 도출할 수 있다. 이때, `granger_causality_test`라는 함수는 다음과 구성되어 있다. 그레인저 인과관계에 대한 내용은 `(참고)`를 참조.

> *<p style="font-size:15px">`wald, pval = bd.ts.test.granger_causality_test(y1, y2, lag)`</p>*
> *<p style="font-size:15px">그레인저 인과관계 검정(Granger Causality Test)</p>*
> - **입력**:
</br>
    y1: 그레인져 인과되는 변수
    </br>
    y2: 그레인저 인과하는 변수
    </br>
    lag: VAR(p) 모형의 최적 시차
    </br>
> - **출력**:
</br>
    wald: Wald 검정 통계량
    </br>
    pval: p-value(틀린 기각의 확률)
    </br>

### (예시)
특정 변수 집합이 다른 변수 집합들을 그레인저 인과하는지 검정하는 예시는 다음과 같다. 우선, 그레인저 인과되는 것으로 예상되는 변수 `y1`가 인플레이션율, 그레인저 인과하는 것으로 예상되는 변수 `y2`가 실질 GDP 상정률과 CD91일 이자율이라고 하자. 나아가 추정된 VAR 모형의 최적 시차를 `lag=2`이라고 하자. 그러면 `granger_causality_test`라는 코드를 사용하여 다음과 같이 그레인저 인과관계 검정을 진행할 수 있다.

`granger_causality_test` 함수를 `gc_test`로 불러온다.

In [3]:
from bok_da.ts.test import granger_causality_test as gc_test

y1과 y2를 설정하고, 그렌져 인과관계 검정을 수행한다.

In [4]:
y1 = df[['inf']]
y2 = df[['rgdp', 'cd91']]

wald, pval = gc_test(y1, y2, lag=2)

H0: ['rgdp' 'cd91'](y2)가 ['inf'](y1)을 그레인져 인과 (Granger Cause)하지 않는다.
 
The p-value is 0.1739


추정된 결과를 해석하면 Wald 검정 통계량을 사용한 p-값 (참인 귀무가설을 잘못 기각시킬 확률)이 약 0.17으로 5\%의 유의수준 하에서`y2`가 `y1`을 그레인저 인과하지 않는다는 귀무가설을 기각하지 못한다. 따라서 통계적으로 5\%의 유의수준 하에서 실질 GDP 성장률과 CD91일 이자율이 인플레이션을 그레인저 인과하지 않는다고 볼 수 있다.

#### (참고)
그레인저 인과관계 (Granger Causality)의 기본 아이디어는 "원인"이 항상 "결과"를 선행한다는 것이다. 다시 말해, 만약 $x_{t}$가 $z_{t}$를 인과하고 있다면, $x_{t}$는 $z_{t}$보다 항상 먼저 일어날 것이며, 더 나아가 $z_{t}$를 예측하는데 있어서 $x_{t}$의 역사를 아는 것이 도움이 된다는 것이다. 이러한 아이디어에 기반하여 그레인저 인과관계는 예측오차의 분산을 완전한 정보 하에서 도출한 경우와 특정 변수들을 제외하는 분완전한 정보 하에서 도출한 경우를 비교한다. 만약 두 경우의 예측오차의 분산이 동일하다면, 제외된 변수들이 목적이 되는 변수들을 예측하는데 있어 어떠한 정보도 주지 못한다고 해석할 수 있다.

> **<p style="font-size:15px">정의 1</p>**
> 어떠한 확률과정 $x_{t}$는 다음과 같은 경우에 $z_{t}$를 그레인져 인과한다고 한다.
> $\begin{equation}\Sigma_{z}(h|\Omega_{t})<\Sigma_{z}(h|\Omega_{t}\setminus\{x_{s},s\leq t\})\notag\end{equation}$
> 말로 풀어서 쓴다면, 완전한 정보 하에서 예측의 목적이 되는 변수인 $z_{t}$의 $h$기 이후 예측에 대한 예측오차의 분산이 $x_{t}$라는 변수의 역사가 주어지지 않은 불완전한 정보 하에서의 예측오차의 분산보다 작다는 것이다.

다음과 같은 이변수 모형에서 그레인저 인과관계를 정의할 수 있다.
> **<p style="font-size:15px">정리 1</p>**
> 어떠한 확률과정 $y_{t}$가 다음과 같은 이변수 모형의 형태를 가진다고 가정하자.
> $\begin{equation}y_{t} = \begin{pmatrix}
        z_{t} \\
        x_{t}
    \end{pmatrix} = \begin{pmatrix}
        \mu_{1} \\
        \mu_{2}
    \end{pmatrix} + \begin{pmatrix}
        \Phi_{11}(L) & 0 \\
        \Phi_{21}(L) & \Phi_{22}(L)
    \end{pmatrix}\begin{pmatrix}
        u_{1t} \\
        u_{2t}
    \end{pmatrix}\notag\end{equation}$
> 그러면, $x_{t}$는 $z_{t}$를 그레인져 인과하지 않는다. 이를 수식으로 풀어쓴다면,
> $\begin{equation}\Sigma_{z}(h|\{y_{s},s\leq t\}) = \Sigma_{z}(h|\{z_{s},s\leq t\}) \iff \Phi_{12}(L)=0 \notag\end{equation}$
> 말로 풀어서 쓴다면, 전체 $y_{t}$를 확률 과정이 모두 주어졌을 경우의 예측오차는 $z_{t}$라는 $y_{t}$의 특정 부분만 주어진 불완전한 정보 하에서의 경우와 동일하며, 이는 계수행렬의 우상방 블록 (upper-right block)이 영행렬로 주어진 것을 통해 이변수 모형 시스템에 반영된다.

> **<p style="font-size:15px">따름 정리 1</p>**
> 어떠한 확률 과정 $y_{t}$가 다음과 같은 축약형 VAR 과정을 따른다고 하자.
> $\begin{align}
    & A(L)y_{t} = \mu + u_{t} \notag \\
    & \iff \begin{pmatrix}
        z_{t} \\
        x_{t}
    \end{pmatrix} = \begin{pmatrix}
        \mu_{1} \\
        \mu_{2}
    \end{pmatrix} + \begin{pmatrix}
        A_{11,1} & A_{12,1} \\
        A_{21,1} & A_{22,1}
    \end{pmatrix}\begin{pmatrix}
        z_{t-1} \\
        x_{t-1}
    \end{pmatrix} + \cdots + \begin{pmatrix}
        A_{11,p} & A_{12,p} \\
        A_{21,p} & A_{22,p}
    \end{pmatrix}\begin{pmatrix}
        z_{t-p} \\
        x_{t-p}
    \end{pmatrix} + \begin{pmatrix}
        u_{1t} \\
        u_{2t}
    \end{pmatrix} \notag
    \end{align}$
> 그러면, $x_{t}$는 $z_{t}$를 그레인져 인과하지 않는다. 이를 수식으로 풀어쓴다면,
> $\begin{equation}
        \Sigma_{z}(h|\{y_{s},s\leq t\}) = \Sigma_{z}(h|\{z_{s},s\leq t\}) \iff A_{12,i}=0\text{ for }i=1,\cdots,p \notag
    \end{equation}$

위 따름 정리의 내용을 사용한다면, 축약형 VAR 모형의 계수행렬의 모든 우측상방 블록이 모두 영행렬인지 확인하는 것을 통해 특정 변수들이 다른 변수들을 그레인져 인과하는지 확인해볼 수 있다. 우선, $N$개의 선형 제약을 사용하여, 그레인져 인과에 대한 귀무가설과 대립가설을 다음과 같이 표현할 수 있다.
$\begin{align}
    & H_{0}\text{: }C\cdot\beta = 0\text{ ; 특정 변수들이 다른 변수들을 그레인저 인과하지 않는다.} \notag \\
    & H_{1}\text{: }C\cdot\beta\neq 0\text{ ; 특졍 변수들이 다른 변수들을 그레인져 인과한다.} \notag
\end{align}$
$\beta = vec(B^{\prime})=vec([v,A_{1},\cdots,A_{p}])$ 일때, VAR 계수 행렬은 다음과 같은 점근적 분포 (asymptotic distribution)를 따른다.
\begin{equation}
    \sqrt{T}(\hat{\beta}-\beta)\to^{d}\mathcal{N}(0,Q^{-1}\otimes\Sigma) \notag
\end{equation}
더 나아가, VAR 모형의 계수 행렬들은 귀무가설 하에서는 다음과 같은 점근적 분포를 따른다는 것을 도출할 수 있다.
\begin{equation}
    \sqrt{T}(C\hat{\beta})\to^{d}\mathcal{N}(0,C[Q^{-1}\otimes\Sigma]C^{\prime}) \notag
\end{equation}
따라서 왈드 검정 통계량 (Wald test statistic)은
\begin{equation}
    Wald = \hat{\beta}^{\prime}C^{\prime}[C[(Z^{\prime}Z)^{-1}\otimes\hat{\Sigma}]C^{\prime}]^{-1}C\hat{\beta} \notag
\end{equation}
이며, 이는 귀무가설 하에서 다음과 같은 카이제곱 분포를 따른다.
\begin{equation}
    Wald\to^{d}\chi_{N}^{2} \notag
\end{equation}